In [1]:
import gspread
import pandas as pd
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import datetime as dt

import pyodbc

In [2]:
# 

In [3]:
MASTER_SHEET_URL = "https://docs.google.com/spreadsheets/d/1fGDYnO2M-PWqMQTBDO78qbgLC-EZKiJIly68_2D-TEA/edit?gid=2047586922#gid=2047586922"
URL_COLUMN_NAME = "Google Sheet Key"   # The column header in your master sheet that contains the URLs
TAB_NAME = "Store Master"

In [4]:

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(r"C:/Users/Administrator/LIS/GCPL SAMT AI/python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

master_sheet = client.open_by_url(MASTER_SHEET_URL)
master_tab = master_sheet.sheet1  # or specify by name: .worksheet("Tab Name")
url_df = pd.DataFrame(master_tab.get_all_records())

In [5]:
url_df.head()

Sr. No. Project Name          SQL Schema Name  \
0        1      Piramal      [PiramalManagement]   
1        2      ZydusGT      [ZydusGTManagement]   
2        3      ZydusMT      [ZydusMTManagement]   
3        4         Amul         [AmulManagement]   
4        5          AWL  [AdaniWilmerManagement]   

                               Google Sheet Key  Active Flag  \
0  1xC719rx6N4mVk97tkXTOyu_fAFM24RFBRBXtptJH9kU            1   
1  1soyCy4I6PHOZPWhyX8w30aSA8vZzMMyK14E3i_mSeGs            1   
2  19chH6DYY_HV4rCaSdxab2IifQYTZ9Db4z_IbotSloN0            1   
3  14wMJ8iak6tInUhO9Qbg8d73WWouW-L0pbf9KY_fRXBY            1   
4  12qp-rP8dRZAoYsy0in2TGZe8OLCWS5rZRK2LfR0mdMo            1   

              Last Run        Last Run Time  
0  2025-07-30 12:40:22  2025-08-04 12:40:17  
1  2025-07-30 12:40:43  2025-08-04 12:40:32  
2  2025-07-30 12:41:20  2025-08-04 12:40:51  
3  2025-07-30 12:41:46  2025-08-04 12:41:03  
4  2025-07-30 12:42:01  2025-08-04 12:41:13

In [6]:
sheet_urls = url_df[URL_COLUMN_NAME].dropna().tolist()

In [7]:
sheet_urls2 = ['https://docs.google.com/spreadsheets/d/1cp3XR718OKo0VAT0G1S_GSotrPgs9BufARBo_riDzBg/edit?gid=569179156#gid=569179156',
              'https://docs.google.com/spreadsheets/d/115JieFKqkiA2OWqjUtv_hAjGAryLrMNKmQEUv8_zPk4/edit?gid=569179156#gid=569179156',
              'https://docs.google.com/spreadsheets/d/14wMJ8iak6tInUhO9Qbg8d73WWouW-L0pbf9KY_fRXBY/edit?gid=1704826248#gid=1704826248',
              'https://docs.google.com/spreadsheets/d/19chH6DYY_HV4rCaSdxab2IifQYTZ9Db4z_IbotSloN0/edit?gid=1704826248#gid=1704826248',
              'https://docs.google.com/spreadsheets/d/1CqQng3BCWTo0HtmIGf4O53x920ZsZycsHJ5_Ijx-W2k/edit?gid=569179156#gid=569179156',
              'https://docs.google.com/spreadsheets/d/1xC719rx6N4mVk97tkXTOyu_fAFM24RFBRBXtptJH9kU/edit?gid=569179156#gid=569179156',
              'https://docs.google.com/spreadsheets/d/1XoR64sjiMRQdPFgowN87KKu9b504xWfibstJ6rXm5TE/edit?gid=569179156#gid=569179156',
            'https://docs.google.com/spreadsheets/d/1igVk60xjFZhM2zr1pR3286m_xKNqtzm-gLKGencCtS4/edit?gid=569179156#gid=569179156',   
               'https://docs.google.com/spreadsheets/d/1vZ2boMDQ4kh3Kvu_dekwa7QH-xfSkInbUfES2G7QpRI/edit?gid=569179156#gid=569179156',
               'https://docs.google.com/spreadsheets/d/1pTpQpFiUuQppHygOfmkoVykUN4NGBGSjs5A7kUcq8k0/edit?gid=569179156#gid=569179156'
              ]

In [8]:
# sheet_urls

In [9]:
compiled_df = pd.DataFrame()

for url in sheet_urls2:
    try:
        sheet = client.open_by_url(url)
        worksheet = sheet.worksheet(TAB_NAME)
        df = get_as_dataframe(worksheet, evaluate_formulas=True, skiprows=0)
        df["Source Sheet"] = sheet.title
        # df['QExecutedTime']=pd.Timestamp.now()
        compiled_df = pd.concat([compiled_df, df], ignore_index=True)
    except Exception as e:
        print(f" Error with {url}: {e}")

# Final cleanup
compiled_df.dropna(how='all', inplace=True)
compiled_df.drop(columns=['From>>','To>>','Visit'],axis=1, inplace=True)

In [10]:
compiled_df.to_csv(r'C:\Users\Administrator\LIS\MISAutomation\Kunal\StoreMaster.csv', index=False)

In [11]:
compiled_df.head()

LIS Store Code Emp User ID Supervisor Name                     Source Sheet
0  SCJStore-1759     SCJ_520      SCJ-SUP-10  Brillon MT Supervisor Dashboard
1   SCJStore-978     SCJ_553      SCJ-SUP-05  Brillon MT Supervisor Dashboard
2  SCJStore-1078     SCJ_552      SCJ-SUP-15  Brillon MT Supervisor Dashboard
3   SCJStore-977     SCJ_471      SCJ-SUP-05  Brillon MT Supervisor Dashboard
4  SCJStore-2223     SCJ_254      SCJ-SUP-05  Brillon MT Supervisor Dashboard

In [12]:
list(compiled_df['Source Sheet'].unique())

['Brillon MT Supervisor Dashboard',
 'GCPL MT Supervisor Dashboard',
 'Amul Supervisor Dashboard',
 'Zydus MT Supervisor Dashboard',
 'Pidilite Supervisor Dashboard',
 'Piramal Supervisor Dashboard',
 'Reckitt Supervisor Dashboard',
 'Wipro Supervisor Dashboard',
 'Parle Supervisor Dashboard',
 'MTR Supervisor Dashboard']

In [13]:
# compiled_df.to_sql(SQL_TABLE_NAME, con=engine, if_exists='replace', index=False)

### Store Masters from SQL Table

In [14]:
server = '157.119.230.120,4071'    
database = 'PowerBI_LISV2'      
username = 'sa'
password = 'Y0m@Sql.!.123'

# Connection string
conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;

"""

In [15]:
conn = pyodbc.connect(conn_str)
final_df =  pd.DataFrame()

In [16]:
x = ['AmulManagement', 'BrillonMTManagement','EvereadyManagement','IDAMManagement','MTRManagement','PidiliteManagement',
     'PiramalManagement','ReckittManagement','WiproGranammaManagement','ZydusMTManagement','HimalayaMTManagement','DanoneMTManagement',
     'GCPLManagement','JohnsonJohnsonManagement','ParleManagement',]

In [17]:
for value in x:
    query = f"""

    SELECT '{value}' as Project,StoreMasterId,ClientStoreCode,LISStoreCode,StoreName,ChainCode,StoreCity,
    StoreLatitude,StoreLongitude,IsEnabled
    FROM [PowerBI_LISV2].[{value}].[StoreMaster] 
    WHERE lower([LISStoreCode]) NOT LIKE '%test%' and IsEnabled='True';
    """

    df = pd.read_sql(query, conn)
    final_df = pd.concat([final_df, df], ignore_index=True)
    print(f"Running for: {value}")

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_29132\172340891.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Running for: AmulManagement
Running for: BrillonMTManagement
Running for: EvereadyManagement
Running for: IDAMManagement
Running for: MTRManagement
Running for: PidiliteManagement
Running for: PiramalManagement
Running for: ReckittManagement
Running for: WiproGranammaManagement
Running for: ZydusMTManagement
Running for: HimalayaMTManagement
Running for: DanoneMTManagement
Running for: GCPLManagement
Running for: JohnsonJohnsonManagement
Running for: ParleManagement


In [18]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61488 entries, 0 to 61487
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Project          61488 non-null  object
 1   StoreMasterId    61488 non-null  int64 
 2   ClientStoreCode  32628 non-null  object
 3   LISStoreCode     61488 non-null  object
 4   StoreName        61488 non-null  object
 5   ChainCode        61488 non-null  object
 6   StoreCity        61488 non-null  object
 7   StoreLatitude    24152 non-null  object
 8   StoreLongitude   24310 non-null  object
 9   IsEnabled        61488 non-null  object
dtypes: int64(1), object(9)
memory usage: 4.7+ MB


In [19]:
final_df=final_df.drop_duplicates()

In [20]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61488 entries, 0 to 61487
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Project          61488 non-null  object
 1   StoreMasterId    61488 non-null  int64 
 2   ClientStoreCode  32628 non-null  object
 3   LISStoreCode     61488 non-null  object
 4   StoreName        61488 non-null  object
 5   ChainCode        61488 non-null  object
 6   StoreCity        61488 non-null  object
 7   StoreLatitude    24152 non-null  object
 8   StoreLongitude   24310 non-null  object
 9   IsEnabled        61488 non-null  object
dtypes: int64(1), object(9)
memory usage: 4.7+ MB


In [21]:
final_df.to_csv(r'C:\Users\Administrator\LIS\MISAutomation\Kunal\SQLStoreMaster.csv', index=False)